In [3]:
import sys
!{sys.executable} -m pip install wwo_hist

ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "/Users/mkhattar/anaconda3/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/Users/mkhattar/anaconda3/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/mkhattar/anaconda3/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3012, in _parsed_pkg_info
    return self._pkg_info
  File "/Users/mkhattar/anaconda3/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/mk

In [189]:
import numpy as np
import pandas as pd

import wwo_hist
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import math

In [5]:
df = pd.read_csv(
    "simulation_data_v2.csv"
    )
df.head()

,Hour,Point,Energy,Timestamp,Date,Day of Week,Out_of_Office,Presurvey,Weekly_Survey,Before_Game_Baseline,In_Game_Baseline
0,8,0.000000,58.383468,2018-09-20 08:00:00,2018-09-20,Thursday,0,4.82,1.79,8.531474,NaN
1,9,0.000000,88.785622,2018-09-20 09:00:00,2018-09-20,Thursday,0,4.82,1.79,11.331923,NaN
2,10,10.000000,76.836119,2018-09-20 10:00:00,2018-09-20,Thursday,0,4.82,1.79,26.189487,NaN
3,11,10.000000,90.501904,2018-09-20 11:00:00,2018-09-20,Thursday,0,4.82,1.79,29.648204,NaN
4,12,9.871989,53.371114,2018-09-20 12:00:00,2018-09-20,Thursday,0,4.82,1.79,29.592628,NaN


# Adding weather data

We first get the range of timestamps, so we know how much weather data to get

In [6]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Timestamp'].min(), df['Timestamp'].max()

(Timestamp('2018-09-20 08:00:00'), Timestamp('2023-05-03 14:00:00'))

We only want to grab a month's worth of data.

In [7]:
df = df[df['Timestamp'] < pd.Timestamp('2018-10-20 08:00:00')]
df['Timestamp'].min(), df['Timestamp'].max()

(Timestamp('2018-09-20 08:00:00'), Timestamp('2018-10-19 17:00:00'))

In [12]:
frequency = 1
start_date = '20-SEPT-2018'
end_date = '19-OCT-2018'
api_key = 'ac2aff5d06ab4bc2bdb181823201106'
location_list = ['singapore']

import os
file_exists = os.path.exists("singapore.csv")

weather_df = (wwo_hist.retrieve_hist_data(api_key,
                                location_list,
                                start_date,
                                end_date,
                                frequency,
                                location_label = False,
                                export_csv = True,
                                store_df = True)[0] 
                     if not file_exists
                     else pd.read_csv("singapore.csv"))

In [13]:
weather_df.head()

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location
0,2018-09-20 00:00:00,33,25,0.0,8.9,7,72,03:35 PM,03:06 AM,06:55 AM,...,4,29,83,0.0,1010,25,10,26,3,singapore
1,2018-09-20 01:00:00,33,25,0.0,8.9,7,72,03:35 PM,03:06 AM,06:55 AM,...,3,27,84,0.0,1010,25,10,27,2,singapore
2,2018-09-20 02:00:00,33,25,0.0,8.9,7,72,03:35 PM,03:06 AM,06:55 AM,...,3,24,84,0.0,1009,25,10,28,2,singapore
3,2018-09-20 03:00:00,33,25,0.0,8.9,7,72,03:35 PM,03:06 AM,06:55 AM,...,2,22,84,0.0,1009,25,10,30,1,singapore
4,2018-09-20 04:00:00,33,25,0.0,8.9,7,72,03:35 PM,03:06 AM,06:55 AM,...,2,25,83,0.0,1010,25,10,130,1,singapore


### Taking temperature, precipitation, humidity, and joining on the main dataframe.

In [18]:
weather_subset_df = (weather_df[['date_time', 'tempC', 'precipMM', 'humidity']]
                    .rename({"date_time": "Timestamp"}, axis=1))
weather_subset_df['Timestamp'] = pd.to_datetime(weather_subset_df['Timestamp'])
weather_subset_df.head()

,Timestamp,tempC,precipMM,humidity
0,2018-09-20 00:00:00,25,0.0,83
1,2018-09-20 01:00:00,25,0.0,84
2,2018-09-20 02:00:00,25,0.0,84
3,2018-09-20 03:00:00,25,0.0,84
4,2018-09-20 04:00:00,25,0.0,83


In [19]:
df = df.merge(right=weather_subset_df, on="Timestamp", how="left")
df.head()

,Hour,Point,Energy,Timestamp,Date,Day of Week,Out_of_Office,Presurvey,Weekly_Survey,Before_Game_Baseline,In_Game_Baseline,tempC,precipMM,humidity
0,8,0.000000,58.383468,2018-09-20 08:00:00,2018-09-20,Thursday,0,4.82,1.79,8.531474,NaN,31,0.1,61
1,9,0.000000,88.785622,2018-09-20 09:00:00,2018-09-20,Thursday,0,4.82,1.79,11.331923,NaN,33,0.1,52
2,10,10.000000,76.836119,2018-09-20 10:00:00,2018-09-20,Thursday,0,4.82,1.79,26.189487,NaN,33,0.1,54
3,11,10.000000,90.501904,2018-09-20 11:00:00,2018-09-20,Thursday,0,4.82,1.79,29.648204,NaN,32,0.0,56
4,12,9.871989,53.371114,2018-09-20 12:00:00,2018-09-20,Thursday,0,4.82,1.79,29.592628,NaN,32,0.0,58


# Preprocessing

## First Check Stationarity of Time Series

Time series are stationary if they do not have trend or seasonal effects. Summary statistics calculated on the time series are consistent over time, like the mean or the variance of the observations.

When a time series is stationary, it can be easier to model. Statistical modeling methods assume or require the time series to be stationary to be effective.

We use the Dickey-Fuller test to test of our time series is stationary.

### Dickey-Fuller Test

In [20]:
from statsmodels.tsa.stattools import adfuller

In [21]:
X = df['Energy']

In [22]:
result = adfuller(X)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

ADF Statistic: -5.133726
p-value: 0.000012
Critical Values:
	1%: -3.454
	5%: -2.872
	10%: -2.572


Since the p-value is less than 0.05, we can reject the null-hypothesis, and proceed with our analysis.

In [23]:
encoder = LabelEncoder()
df["Day of Week"] = encoder.fit_transform(df["Day of Week"])

## Remove extra columns

In [24]:
df = (df[['Timestamp', 'Point', 'Day of Week', 'Out_of_Office', 'tempC', 'precipMM', 'humidity', 'Energy']]
      .set_index('Timestamp'))
df.head()

,Point,Day of Week,Out_of_Office,tempC,precipMM,humidity,Energy
Timestamp,,,,,,,
2018-09-20 08:00:00,0.000000,4,0,31,0.1,61,58.383468
2018-09-20 09:00:00,0.000000,4,0,33,0.1,52,88.785622
2018-09-20 10:00:00,10.000000,4,0,33,0.1,54,76.836119
2018-09-20 11:00:00,10.000000,4,0,32,0.0,56,90.501904
2018-09-20 12:00:00,9.871989,4,0,32,0.0,58,53.371114


## Check for nan values

In [25]:
df.isnull().values.any()

False

## `series_to_supervised` (Time lag function)

In [168]:
def series_to_supervised(data,
                         n_in,
                         target_features,
                         col_names=None,
                         n_out=1,
                         dropnan=True,
                         initial_excluded_timesteps=0,
                         removed_features=[],
                         only_current_timestep_features=[]):
    """Takes time series data and converts it into a supervised learning
    problem framework.

        Parameters:
            - data (pd.Dataframe) -- the time series data to be converted.
            - n_in (int) -- Number of time steps to use as lag for the feature 
                matrix
            - col_names (List[str]) -- list of strings to use as column names,
                that get converted into features for each time lag
            - target_features (List[str]) -- List of features that will be used
                as dependent variables in the target matrix.
            - n_out (int) -- Number of time steps to use as lag for the target
                matrix
            - dropnan (bool) -- Whether to drop nan values
            - initial_excluded_timesteps (int) -- The number of input timesteps to 
                ignore before starting the time lag.
            - removed_features (List[str]) -- List of features that should be removed
                from the dataframe. 
            - only_current_timestep_features (List[str]) -- Features that should
            only be included in the current timestep, not any before (e.g.) to avoid
            unintended dependencies
            
            - [Planned] exclude_current_day (bool) -- Whether to include values 
                from the current day. If this parameter is false, then the time lag 
                will always start with the day preceding the current time 
                step.
                
        Outputs:
            - (X, y): (Feature matrix, target matrix)
    """
    
    if col_names is None:
        col_names = data.columns
        
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data).drop(removed_features, axis=1)
    col_names = [x for x in col_names if x not in removed_features]
    cols, names = list(), list()      
        
    only_prev_time_features = df.drop(only_current_timestep_features, axis=1)
    only_prev_column_names = [x for x in col_names if x in only_prev_time_features.columns]

    # (t-n, ... t-1) --> i.e. steps into the past
    for i in range(n_in + initial_excluded_timesteps, initial_excluded_timesteps, -1):
        cols.append(only_prev_time_features.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in only_prev_column_names]

    # (t, t+1, ... t+n) --> i.e. steps into the future
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('%s(t)' % (col)) for col in col_names]
        else:
            names += [('%s(t + %d)' % (col, i)) for col in col_names]

    # concat
    agg = pd.concat(cols, axis=1)
#     agg.columns = names

    # dropnan
    if dropnan:
        agg.dropna(inplace=True)
        
    Y_vals_cols = ([('%s(t)' % (col)) for col in target_features] + 
        ['%s(t + %d)' % (col, i) for col in target_features for i in range(1, n_out)])
    
    agg.columns = names
    
    Y_vals = agg[Y_vals_cols]
    X_vals = agg.drop(Y_vals_cols, axis=1)
        
    return X_vals.values, Y_vals.values

## Make a scaled dataframe

In [115]:
df_y = df["Energy"]
df_X = df.drop("Energy", axis = 1)

scaler_X = MinMaxScaler(feature_range = (0, 1))
scaler_y = MinMaxScaler(feature_range = (0, 1))

df_y = df_y.astype("float32")
df_X = df_X.astype("float32")

scaled = pd.DataFrame(scaler_X.fit_transform(df_X), columns = df_X.columns)
scaled_y = scaler_y.fit_transform(values_y.reshape(-1, 1))

scaled['Energy'] = scaled_y

# Benchmark Testing

## Model evaluation metrics

In [29]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

def rmse(predicted_y, actual_y):
    return sqrt(mean_squared_error(actual_y, predicted_y))

def cv_rmse(predicted_y, actual_y):
    """This measures the coefficient of variation, which is the RMSE normalized by 
    the mean of the measured values and quantifies typical size of the error 
    relative to the mean of the observations. A high CV score indicates that a 
    model has a high error range.
    """
    return rmse(predicted_y, actual_y) / sqrt(np.mean(actual_y))

def mae(predicted_y, actual_y):
    return mean_absolute_error(actual_y, predicted_y)

evaluation_methods = [rmse, cv_rmse, mae]

## KNN

## Random Forests

## Gradient Boosting

## Standard NN

## Extra Trees Regressor

# Model Selection

## Genetic algorithm to find optimal amount of time lag

## Exploring AutoML

## Exploring Bayesian Optimization (gyopt)

In [149]:
import GPy, GPyOpt

In [193]:
from sklearn.model_selection import train_test_split

class Network:
    def __init__(self, 
                 network_size, 
                 epochs, 
                 batch_size,
                 number_of_timesteps,
                 num_lstm_layers,
                 validation_split):
        
        self.network_size = network_size
        self.epochs = epochs
        self.batch_size = batch_size
        self.number_of_timesteps = number_of_timesteps
        self.num_lstm_layers = num_lstm_layers
        self.validation_split = validation_split
        
        self.X_train, self.X_test, self.y_train, self.y_test = self.network_data()
        self.model = self.lstm_model()
        
   # load network data
    def network_data(self):
        supervised_X, supervised_y = series_to_supervised(data = scaled, target_features=['Energy'], 
                                only_current_timestep_features=['Point'],
                                initial_excluded_timesteps=10,
                                col_names = df.columns.tolist(), 
                                n_in = self.number_of_timesteps,
                                n_out = 1)
        X_train, X_test, y_train, y_test = train_test_split(supervised_X, supervised_y, 
                                                            test_size=0.33, random_state=42)
        
        X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
        
        return X_train, X_test, y_train, y_test
    
    def lstm_model(self):
        model = Sequential()
        for _ in range(self.num_lstm_layers - 1):
            model.add(LSTM(self.network_size, input_shape = (self.X_train.shape[1], self.X_train.shape[2]), 
                           return_sequences = True))
        model.add(LSTM(128, return_sequences = False))
        model.add(Dense(1))
        model.compile(loss="mse", optimizer="adam")
        
        return model
    
    def model_fit(self):
        early_stopping = EarlyStopping(patience=0, verbose=1)
        # fit network
        self.model.fit(
            self.X_train,
            self.y_train,
            epochs=self.epochs,
            batch_size=self.batch_size,
            validation_split=self.validation_split,
            verbose=0,
            callbacks=[early_stopping],
        )
        
    def model_evaluate(self):
        self.model_fit()
    
#         evaluation = self.model.evaluate(self.X_test, self.y_test, batch_size=self.batch_size, verbose=0)
        yhat = self.model.predict(self.X_test)
        # %%
        # invert scaling for forecast
        inv_yhat = scaler_y.inverse_transform(yhat)
        inv_yhat = inv_yhat[:, -1]

        #%%
        # invert scaling for actual
        temp_y_test = self.y_test.reshape((len(self.y_test), 1))
        inv_y = scaler_y.inverse_transform(temp_y_test)
        inv_y = inv_y[:, -1]

        # calculate RMSE
        
        rmse = math.sqrt(mean_squared_error(inv_y, inv_yhat))
        return rmse

In [194]:
def run_lstm(network_size=128, 
             epochs=50, 
             batch_size=256,
             number_of_timesteps=20,
             num_lstm_layers=2,
             validation_split=0.1):
    
    _model = Network(network_size, 
                     epochs, 
                     batch_size,
                     number_of_timesteps,
                     num_lstm_layers,
                     validation_split)
    model_evaluation = _model.model_evaluate()
    return model_evaluation

In [195]:
run_lstm()

Epoch 00007: early stopping


130.01043377600507

In [196]:
# the bounds dict should be in order of continuous type and then discrete type
bounds = [{'name': 'validation_split', 'type': 'continuous',  'domain': (0.0, 0.3)},
          {'name': 'network_size', 'type': 'discrete',  'domain': (128, 256, 512)},
        {'name': 'epochs',           'type': 'discrete',    'domain': (5, 10, 20, 50)},
          {'name': 'batch_size',       'type': 'discrete',    'domain': (10, 100, 500)},
          {'name': 'number_of_timesteps', 'type': 'discrete',  'domain': (5, 10, 20, 40)},
          {'name': 'num_lstm_layers', 'type': 'discrete',  'domain': (1, 2, 3, 4)}]

In [197]:
def f(x):
    print(x)
    evaluation = run_lstm(
        network_size=int(x[:,1]), 
        epochs = int(x[:,2]), 
        batch_size = int(x[:,3]),
        number_of_timesteps = int(x[:,4]), 
        num_lstm_layers = int(x[:,5]),  
        validation_split = float(x[:,0]))
    print("LOSS:\t{0}".format(evaluation))
    print(evaluation)
    return evaluation

In [198]:
opt_lstm = GPyOpt.methods.BayesianOptimization(f=f, domain=bounds)

[[2.72436498e-01 1.28000000e+02 5.00000000e+00 5.00000000e+02
  2.00000000e+01 3.00000000e+00]]
LOSS:	213.55825646939994
213.55825646939994
[[1.49467966e-01 2.56000000e+02 2.00000000e+01 1.00000000e+02
  4.00000000e+01 3.00000000e+00]]
Epoch 00004: early stopping
LOSS:	133.0501109779695
133.0501109779695
[[1.95771841e-01 2.56000000e+02 5.00000000e+01 1.00000000e+02
  2.00000000e+01 4.00000000e+00]]
Epoch 00006: early stopping
LOSS:	142.58215148424082
142.58215148424082
[[  0.16646716 128.          20.          10.          20.
    4.        ]]
Epoch 00008: early stopping
LOSS:	80.81843015268083
80.81843015268083
[[  0.28917712 128.           5.          10.          20.
    4.        ]]
Epoch 00002: early stopping
LOSS:	148.26717201348046
148.26717201348046


In [199]:
opt_lstm.run_optimization(max_iter=10)

[[  0.3 128.   20.   10.   20.    4. ]]
Epoch 00002: early stopping
LOSS:	149.2208867648058
149.2208867648058
[[  0.13321274 128.          20.          10.          20.
    4.        ]]
Epoch 00006: early stopping
LOSS:	87.91811931066883
87.91811931066883
[[1.22709434e-01 1.28000000e+02 2.00000000e+01 1.00000000e+01
  2.00000000e+01 3.00000000e+00]]
Epoch 00006: early stopping
LOSS:	85.53583975628287
85.53583975628287
[[2.96962774e-01 5.12000000e+02 5.00000000e+01 1.00000000e+02
  4.00000000e+01 4.00000000e+00]]
Epoch 00003: early stopping
LOSS:	132.2974498407849
132.2974498407849
[[  0.1871012 128.         20.         10.         20.          3.       ]]
Epoch 00004: early stopping
LOSS:	103.64970208441267
103.64970208441267
[[4.67018922e-02 1.28000000e+02 2.00000000e+01 1.00000000e+01
  2.00000000e+01 3.00000000e+00]]
Epoch 00006: early stopping
LOSS:	83.03205859128298
83.03205859128298
[[2.00264212e-02 1.28000000e+02 2.00000000e+01 1.00000000e+01
  2.00000000e+01 2.00000000e+00]]
Ep

In [200]:
print("""
Optimized Parameters:
\t{0}:\t{1}
\t{2}:\t{3}
\t{4}:\t{5}
\t{6}:\t{7}
\t{8}:\t{9}
\t{10}:\t{11}
""".format(bounds[0]["name"],opt_lstm.x_opt[0],
           bounds[1]["name"],opt_lstm.x_opt[1],
           bounds[2]["name"],opt_lstm.x_opt[2],
           bounds[3]["name"],opt_lstm.x_opt[3],
           bounds[4]["name"],opt_lstm.x_opt[4],
           bounds[5]["name"],opt_lstm.x_opt[5],))
print("optimized loss: {0}".format(opt_lstm.fx_opt))


Optimized Parameters:
	validation_split:	0.0009935909309099415
	network_size:	128.0
	epochs:	20.0
	batch_size:	10.0
	number_of_timesteps:	20.0
	num_lstm_layers:	1.0

optimized loss: 79.0532881244117


# GA-augmented LSTM-RNN model

# LSTM-GAN Testing